In [ ]:
import torch
import math
x = torch.tensor([[1/3,1/3,1/3],[1.,.0,.0],[.1,.4,.5],[.1,.1,.8]], dtype=torch.float32)
batch_size = 4
label_size = 3
threshold = 0.75

In [ ]:

entr = torch.ones(batch_size, requires_grad=False) #[batch_size] 
pred = torch.zeros((batch_size,label_size), requires_grad=False) #[batch_size, label_size]
total_remain_idx = torch.ones(batch_size, dtype=torch.bool) #[batch_size]

In [ ]:
x_out = x
#for idx in range(self.num_layer):
for idx in range(5):
    # f forward
    #x_out = self.layer_forward(x_out, idx, mask)
    # return form b/h
    #y_out = self.bridge_return(x_out, idx, mask)
    y_out = torch.rand(x_out.size())

    y_entr = torch.sum(torch.special.entr(y_out),dim=-1) / math.log(y_out.size(-1))

    total_remain_idx = entr>threshold
    entr[total_remain_idx] = y_entr
    pred[total_remain_idx,:] = y_out
    
    remain_idx = y_entr>threshold
    print(f"\n{idx}")
    print("total",total_remain_idx)
    print("remain",remain_idx)
    print("entr",entr)
    print("pred",pred)
    print("x",x_out)
    x_out = x_out[remain_idx,:]
    
    
    print("x_remain",x_out)
    if x_out.size(0) == 0:
        break


In [ ]:
y_out[y_entr>threshold]

In [ ]:
total_remain_idx = torch.ones(batch_size, dtype=torch.bool)


In [ ]:
total_remain_idx[[True,False,False,False]].size(0) == 0

In [ ]:
from datasets import load_dataset
train_data = load_dataset('dbpedia_14', split='train')
valid_data = load_dataset('dbpedia_14', split='test[:50%]')
test_data = load_dataset('dbpedia_14', split='test[50%:]')

In [ ]:
from datasets import load_dataset
train_data = load_dataset('dbpedia_14', split='train')
test_data = load_dataset('dbpedia_14', split='test').shuffle(seed=35)


split_size = int(test_data.num_rows*0.5)
valid_data = test_data[:split_size]
test_data = test_data[split_size:]

train_text = [b['content'] for b in train_data]
train_label = [b['label'] for b in train_data]

In [ ]:
type(test_data['label'])

In [ ]:
len(test_data['label'])

for i in test_data.keys():
    print(i)







In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

class_num = 2
df = pd.read_csv('./IMDB_Dataset.csv')
df['cleaned_reviews'] = df['review']
corpus = [word for text in df['cleaned_reviews'] for word in text.split()]
text = [t for t in df['cleaned_reviews']]
label = []
for t in df['sentiment']:
    if t == 'negative':
        label.append(1)
    else:
        label.append(0)

clean_train, clean_test, train_label, test_label = train_test_split(text, label, test_size=0.2, random_state=35)
clean_valid, clean_test, valid_label, test_label = train_test_split(clean_test, test_label, test_size=0.5, random_state=35)

In [ ]:
import pandas
print(pandas.DataFrame(train_label).value_counts())
print(pandas.DataFrame(valid_label).value_counts())
print(pandas.DataFrame(test_label).value_counts())
print(len(clean_train))
print(len(clean_valid))
print(len(clean_test))
print(len(train_label))
print(len(valid_label))
print(len(test_label))

In [20]:
import torch
from torchvision import transforms, datasets


class TwoCropTransform:
    """Create two crops of the same image"""
    def __init__(self, transform1, transform2=None):
        self.transform1 = transform1
        if transform1 != None and transform2 == None:
            self.transform2 = transform1
        else:
            self.transform2 = transform2

    def __call__(self, x):
        if self.transform1 == None:
            return [x, x]
        return [self.transform1(x), self.transform2(x)]

def set_loader(dataset, train_bsz, test_bsz, augmentation_type):
    if dataset == "cifar10":
        n_classes = 10
        mean = (0.4914, 0.4822, 0.4465)
        std = (0.2023, 0.1994, 0.2010)
    elif dataset == "cifar100":
        n_classes = 100
        mean = (0.5071, 0.4867, 0.4408)
        std = (0.2675, 0.2565, 0.2761)
    elif dataset == "tinyImageNet":
        n_classes = 200
    else:
        raise ValueError("Dataset not supported: {}".format(dataset))
    
    if dataset == "cifar10" or dataset == "cifar100":
        normalize = transforms.Normalize(mean=mean, std=std)
        weak_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])

        strong_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.RandomApply([
                transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)
            ], p=0.8),
            transforms.RandomGrayscale(p=0.2),
            transforms.ToTensor(),
            normalize,
        ])

        test_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])

    if dataset == "tinyImageNet":
        normalize = transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2770, 0.2691, 0.2821])

        
        weak_transform = transforms.Compose([
        transforms.Resize(32),
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
        ])

        strong_transform = transforms.Compose([
            transforms.Resize(32),
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.RandomApply([
                transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)
            ], p=0.8),
            transforms.RandomGrayscale(p=0.2),
            transforms.ToTensor(),
            normalize,
        ])

        test_transform = transforms.Compose([
            transforms.Resize(32),
            transforms.ToTensor(),
            normalize
        ])

    if augmentation_type == "basic":
        source_transform = weak_transform
        target_transform = None
    elif augmentation_type == "strong":
        source_transform = TwoCropTransform(weak_transform, strong_transform)
        target_transform = TwoCropTransform(None)
    else:
        raise ValueError("Augmentation type not supported: {}".format(augmentation_type))


    if dataset == "cifar10":
        train_set = datasets.CIFAR10(root='./cifar10', transform=source_transform, target_transform = target_transform,  download=True)
        test_set = datasets.CIFAR10(root='./cifar10', train=False, transform=test_transform)
    elif dataset == "cifar100":
        train_set = datasets.CIFAR100(root='./cifar100', transform=source_transform, target_transform = target_transform, download=True)
        test_set = datasets.CIFAR100(root='./cifar100', train=False, transform=test_transform)
    elif dataset == "tinyImageNet":
        train_set = datasets.ImageFolder('./tiny-imagenet-200/train', transform=source_transform)
        test_set = datasets.ImageFolder('./tiny-imagenet-200/val', transform=test_transform)


    print(type(train_set))
    print(len(train_set))
    
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=train_bsz, shuffle=True, pin_memory=True, num_workers=4)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=test_bsz, shuffle=False, pin_memory=True)
    

    return train_loader, test_loader, n_classes

configs = {
    'dataset' : "tinyImageNet",
    'train_bsz': 128, 
    'test_bsz': 128, 
    'aug_type': 'strong',
}
train_loader, test_loader, n_classes = set_loader(configs['dataset'], configs['train_bsz'], configs['test_bsz'], configs['aug_type'])

<class 'torchvision.datasets.folder.ImageFolder'>
100000


In [21]:
for X, Y in train_loader:
    X = torch.cat(X)
    Y = torch.cat([Y, Y])
    print(X.size())
    print(Y.size())

torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([256, 3, 32, 32])
torch.Size([256])
torch.Size([2